In [28]:
import community
import pandas as pd
from pandas import DataFrame as df
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import random
import time
import collections

# Ignore matplotlib warnings
import warnings
from scipy.sparse import (spdiags, SparseEfficiencyWarning, csc_matrix,
    csr_matrix, isspmatrix, dok_matrix, lil_matrix, bsr_matrix)
warnings.simplefilter('ignore',SparseEfficiencyWarning)
data = pd.read_csv("visualisasi_karate.csv")

In [29]:
def pembangkitan(g):
    pop_size = max(g.nodes())
    pembangkitan = []

    while len(pembangkitan) < pop_size:
        a = round(np.random.uniform(low=min(g.nodes()), high=pop_size))
        if a not in pembangkitan:
            pembangkitan.append(a)
    return pembangkitan

def merge(lsts):
    sets = [set(lst) for lst in lsts if lst]
    merged = True
    while merged:
        merged = False
        results = []
        while sets:
            common, rest = sets[0], sets[1:]
            sets = []
            for x in rest:
                if x.isdisjoint(common):
                    sets.append(x)
                else:
                    merged = True
                    common |= x
            results.append(common)
        sets = results
    return sets

def comm(G, pembangkitan, node):
    mrg = []
    for index, i in enumerate(pembangkitan):
        temp = []
        for j in [ u for u in G.neighbors(i)]:
            temp.append(j)
        temp.append(i)
        mrg.append(set(temp))
    mrg = list(set(map(tuple, mrg)))
    mrg.sort(key=len, reverse = True)

    final = merge(mrg)

    for i in range(len(final)):
        lc = dict()
        for j in final[i]:
            lc[j] = (G.degree(j)/(100/50))
        final[i] = lc
        
    for i in range(len(final)):
        for k,v in final[i].items():
            if node in final[i].keys():
                return i
            
def lc(G, pembangkitan, node):
    mrg = []
    for index, i in enumerate(pembangkitan):
        temp = []
        for j in [ u for u in G.neighbors(i)]:
            temp.append(j)
        temp.append(i)
        mrg.append(set(temp))
    mrg = list(set(map(tuple, mrg)))
    mrg.sort(key=len, reverse = True)

    final = merge(mrg)

    for i in range(len(final)):
        lc = dict()
        for j in final[i]:
            lc[j] = (G.degree(j)/(100/50))
        final[i] = lc
        
    for i in range(len(final)):
        for k,v in final[i].items():
            if node in final[i].keys():
                return int(sum(final[i].values()))

def Q_xi(G, pembangkitan):
    tabel = dict()
    for index, node in enumerate(pembangkitan):
        try:
            tabel[index] = [node, G.degree(node), 
                    G.number_of_edges(), 
                    lc(G, pembangkitan, node), 
                    lc(G, pembangkitan, node)*2, 
                    comm(G, pembangkitan, node),
                    (lc(G, pembangkitan, node)/G.number_of_edges())-((lc(G, pembangkitan, node)*2)/(2*G.number_of_edges()))**(2)]
        except ZeroDivisionError:
            tabel[index] = [node, G.degree(node), 
                    G.number_of_edges(), 
                    lc(G, pembangkitan, node), 
                    lc(G, pembangkitan, node)*2, 
                    comm(G, pembangkitan, node),
                    0]
        
    return tabel

def Qbest(G, pembangkitan):
    tabel = Q_xi(G, pembangkitan)
    max_com = [0]
    for i in range(len(tabel)):
        max_com.append((tabel[i])[5])

    Q = dict()    
    for i in range(max(max_com)+1):
        Qi = dict()
        for j in range(len(tabel)):
            if (tabel[j])[5] == i:
                Qi[j] = (tabel[j])[6]
        try:
            Q[i] = sum(Qi.values())/len(Qi)
        except ZeroDivisionError:
            Q[i] = 0
        
    return sum(Q.values())

In [ ]:
g = nx.from_pandas_edgelist(data, source='source', target='target', edge_attr=None)
d = dict(g.degree)
community = dict()
for i in g.nodes():
    color = ["#FF0000","#FFFF00","#008000","#0000FF","#FF00FF","#808000","#00FF00","#00FFFF","#800000","#800080"]
    community[i] = color[comm(g, pembangkitan(g), i)]
plt.figure(10,figsize=(15,10))
nx.draw_spring(g, with_labels = True, node_size=[v * 300 for v in d.values()], node_color=community.values())
plt.savefig('karate.svg')

In [27]:
for i in g.nodes():
    print(i, comm(g, pembangkitan(g), i))

2 0
12 0
17 0
10 0
1 4
27 4
3 2
24 2
15 2
29 2
34 2
11 2
4 3
21 3
23 3
9 3
5 0
19 0
6 0
20 0
7 1
14 1
30 1
18 1
8 1
33 1
26 1
25 0
13 1
22 1
31 0
16 1
28 1
32 1


In [15]:
karate = [[2,12,17,10,5,19,6,20,25,31],[7,14,30,18,8,33,26,13,22,16,28,32],[3,24,15,29,34,11],[4,21,23,9],[1,27]]
g = nx.from_pandas_edgelist(data, source='source', target='target', edge_attr=None)
nx.algorithms.community.quality.modularity(g,karate)

-0.09475673898750805

In [30]:
g = nx.from_pandas_edgelist(data, source='source', target='target', edge_attr=None)

print(Q_xi(g,pembangkitan(g)))

{0: [16, 1, 46, 19, 38, 1, 0.24243856332703215], 1: [24, 1, 46, 6, 12, 2, 0.11342155009451796], 2: [13, 4, 46, 19, 38, 1, 0.24243856332703215], 3: [26, 5, 46, 19, 38, 1, 0.24243856332703215], 4: [17, 6, 46, 16, 32, 0, 0.22684310018903592], 5: [25, 3, 46, 16, 32, 0, 0.22684310018903592], 6: [27, 1, 46, 1, 2, 4, 0.021266540642722116], 7: [20, 4, 46, 16, 32, 0, 0.22684310018903592], 8: [22, 5, 46, 19, 38, 1, 0.24243856332703215], 9: [28, 2, 46, 19, 38, 1, 0.24243856332703215], 10: [30, 3, 46, 19, 38, 1, 0.24243856332703215], 11: [29, 2, 46, 6, 12, 2, 0.11342155009451796], 12: [23, 2, 46, 4, 8, 3, 0.07939508506616257], 13: [8, 2, 46, 19, 38, 1, 0.24243856332703215], 14: [32, 1, 46, 19, 38, 1, 0.24243856332703215], 15: [19, 4, 46, 16, 32, 0, 0.22684310018903592], 16: [6, 3, 46, 16, 32, 0, 0.22684310018903592], 17: [5, 2, 46, 16, 32, 0, 0.22684310018903592], 18: [4, 3, 46, 4, 8, 3, 0.07939508506616257], 19: [18, 3, 46, 19, 38, 1, 0.24243856332703215], 20: [14, 3, 46, 19, 38, 1, 0.24243856332

In [31]:
print(Qbest(g,pembangkitan(g)))

0.6833648393194707
